In [1]:
import cv2
import numpy as np
from apriltag import apriltag

In [3]:
#april tag detection    
import cv2
import numpy as np
import os
from dt_apriltags import Detector

# Load camera calibration data
root = './'  # Adjust as needed
calib_data_path = os.path.join(root, 'charuco_calib', 'calib_data', 'MultiMatrix.npz')
calib_data = np.load(calib_data_path)

cam_mat = calib_data["camMatrix"]
dist_coef = calib_data["distCoef"]
camera_params = [cam_mat[0, 0], cam_mat[1, 1], cam_mat[0, 2], cam_mat[1, 2]]

img_path = os.path.join(root, 'apriltag-imgs')
# Initialize the AprilTag detector
at_detector = Detector(
    searchpath=img_path,
    families='tag36h11',     # Tag family
    nthreads=4,              # Use multiple threads for faster processing
    quad_decimate=2.0,       # Downsampling factor for faster detection
    quad_sigma=0.8,          # Apply blur to reduce noise
    refine_edges=True,       # Snap edges to improve accuracy
    decode_sharpening=0.25,  # Sharpen the decoded tag       
)

# # Start webcam feed
cap = cv2.VideoCapture(2)  # Change index if needed
cv2.namedWindow('AprilTag Detection', cv2.WINDOW_NORMAL)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect AprilTags
    tags = at_detector.detect(gray, estimate_tag_pose=True, camera_params=camera_params, tag_size=0.05)
    
    for tag in tags:
        # Draw bounding box
        for i in range(4):
            pt1 = (int(tag.corners[i][0]), int(tag.corners[i][1]))
            pt2 = (int(tag.corners[(i + 1) % 4][0]), int(tag.corners[(i + 1) % 4][1]))
            cv2.line(frame, pt1, pt2, (0, 255, 0), 2)

        # Draw tag center
        center = (int(tag.center[0]), int(tag.center[1]))
        cv2.circle(frame, center, 5, (255, 0, 0), -1)
        
        # Display tag ID and distance
        cv2.putText(frame, f"ID: {tag.tag_id}", (center[0] + 10, center[1] + 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
        if tag.pose_R is not None and tag.pose_t is not None:
            # Distance from camera in meters
            dist = np.linalg.norm(tag.pose_t)
            cv2.putText(frame, f"Dist: {dist:.2f} m", (center[0] + 10, center[1] + 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
            
    
    # # Display the frame
    cv2.imshow('AprilTag Detection', frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x60283efe2d10) is not the object's thread (0x60283f97e900).
Cannot move to target thread (0x60283efe2d10)

QObject::moveToThread: Current thread (0x60283efe2d10) is not the object's thread (0x60283f97e900).
Cannot move to target thread (0x60283efe2d10)

QObject::moveToThread: Current thread (0x60283efe2d10) is not the object's thread (0x60283f97e900).
Cannot move to target thread (0x60283efe2d10)

QObject::moveToThread: Current thread (0x60283efe2d10) is not the object's thread (0x60283f97e900).
Cannot move to target thread (0x60283efe2d10)

QObject::moveToThread: Current thread (0x60283efe2d10) is not the object's thread (0x60283f97e900).
Cannot move to target thread (0x60283efe2d10)

QObject::moveToThread: Current thread (0x60283efe2d10) is not the object's thread (0x60283f97e900).
Cannot move to target thread (0x60283efe2d10)

QObject::moveToThread: Current thread (0x60283efe2d10) is not the object's thread (0x60283f97e900).
Cannot move to tar

In [ ]:
#apriltag - midas - normalized depth
import cv2
import torch
import numpy as np
from dt_apriltags import Detector

# Load MiDaS model
model_type = "MiDaS_small"
midas = torch.hub.load("intel-isl/MiDaS", model_type).to("cuda")
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.small_transform if model_type == "MiDaS_small" else midas_transforms.dpt_transform

# Load camera calibration data
calib_data = np.load("charuco_calib/calib_data/MultiMatrix.npz")
cam_mat = calib_data["camMatrix"]
dist_coef = calib_data["distCoef"]
camera_params = [cam_mat[0, 0], cam_mat[1, 1], cam_mat[0, 2], cam_mat[1, 2]]

# AprilTag parameters
TAG_SIZE = 0.05  # meters (4.6 cm)
at_detector = Detector(
    families='tag36h11',
    nthreads=4,
    quad_decimate=2.0,
    quad_sigma=0.8,
    refine_edges=True,
    decode_sharpening=0.25
)

# Start video capture
cap = cv2.VideoCapture(2)

scale_factor = 1.0  # Default scale factor for relative depth

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # frame = cv2.resize(frame, (256, 256))
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute MiDaS depth first
    input_batch = transform(frame).to("cuda")
    with torch.no_grad():
        prediction = midas(input_batch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1), size=frame.shape[:2], mode="bicubic", align_corners=False
        ).squeeze()

    output = prediction.cpu().numpy()  # Relative depth

    # Detect AprilTags
    tags = at_detector.detect(gray_frame, estimate_tag_pose=True, camera_params=camera_params, tag_size=TAG_SIZE)
    
    for tag in tags:
        # Draw bounding box
        for i in range(4):
            pt1 = (int(tag.corners[i][0]), int(tag.corners[i][1]))
            pt2 = (int(tag.corners[(i + 1) % 4][0]), int(tag.corners[(i + 1) % 4][1]))
            cv2.line(frame, pt1, pt2, (0, 255, 0), 2)

        # Draw tag center
        center = (int(tag.center[0]), int(tag.center[1]))
        cv2.circle(frame, center, 5, (255, 0, 0), -1)

        # Display tag ID
        cv2.putText(frame, f"ID: {tag.tag_id}", (center[0] + 10, center[1] + 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

        if tag.pose_t is not None:
            # Absolute distance from camera in meters
            distance = np.linalg.norm(tag.pose_t)
            cv2.putText(frame, f"Dist: {distance:.2f} m", (center[0] + 10, center[1] + 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

            # Get bounding box coordinates for marker region
            x_min = int(min(tag.corners[:, 0]))
            y_min = int(min(tag.corners[:, 1]))
            x_max = int(max(tag.corners[:, 0]))
            y_max = int(max(tag.corners[:, 1]))

            # Extract depth values in marker region
            marker_depth_relative = np.median(output[y_min:y_max, x_min:x_max])

            # Compute scale factor
            if marker_depth_relative > 0:
                scale_factor = distance / marker_depth_relative

    # Convert MiDaS depth to absolute depth using scale factor
    output_absolute = output * scale_factor

    # Normalize for visualization
    output_absolute = cv2.normalize(output_absolute, None, 0, 255, cv2.NORM_MINMAX, cv2.cv2_8U)
    # output_colored = cv2.applyColorMap(output_absolute, cv2.COLORMAP_JET)

    # Display depth map and frame
    cv2.imshow("Depth Map", output_absolute)
    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) == ord("q"):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


Using cache found in /home/abhinand/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /home/abhinand/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /home/abhinand/.cache/torch/hub/intel-isl_MiDaS_master
QObject::moveToThread: Current thread (0x577419f50e90) is not the object's thread (0x57741b0fe400).
Cannot move to target thread (0x577419f50e90)

QObject::moveToThread: Current thread (0x577419f50e90) is not the object's thread (0x57741b0fe400).
Cannot move to target thread (0x577419f50e90)

QObject::moveToThread: Current thread (0x577419f50e90) is not the object's thread (0x57741b0fe400).
Cannot move to target thread (0x577419f50e90)

QObject::moveToThread: Current thread (0x577419f50e90) is not the object's thread (0x57741b0fe400).
Cannot move to target thread (0x577419f50e90)

QObject::moveToThread: Current thread (0x577419f50e90) is not the object's thread (0x57741b0fe400).
Cannot move to target thread (0x577419f50e90)

QObject::moveToThread: Current thread (0x577419f50e90) is not the object's thread (0x57741b0fe

In [5]:
#apriltag - midas - rect depth
import os
import cv2
import numpy as np
import torch
from torchvision.transforms import transforms
from dt_apriltags import Detector
import mediapipe as mp

# Load camera calibration data
root = os.getcwd()
calib_data_path = os.path.join(root, 'charuco_calib', 'calib_data', 'MultiMatrix.npz') 
calib_data = np.load(calib_data_path)
cam_mat = calib_data["camMatrix"]
dist_coef = calib_data["distCoef"]

# AprilTag setup
TAG_SIZE = 0.05  # meters (5 cm)
at_detector = Detector(
    families="tag36h11",
    nthreads=1,
    quad_decimate=1.0,
    quad_sigma=0.0,
    refine_edges=1,
    decode_sharpening=0.25,
    debug=0
)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Load MiDaS model
model_type = "MiDaS_small"
midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.small_transform if model_type == "MiDaS_small" else midas_transforms.dpt_transform

def estimate_relative_depth(frame):
    """Estimate relative depth using MiDaS and resize to original frame size."""
    input_batch = transform(frame).to(device)
    with torch.no_grad():
        prediction = midas(input_batch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1), size=frame.shape[:2], mode="bicubic", align_corners=False
        ).squeeze()
    output = prediction.cpu().numpy()
    output = 1.0 / (output + 1e-6)  # Invert depth map
    return output

# Smoothing factor for fingertip positions
SMOOTHING_FACTOR = 0.3
last_positions = {}

cap = cv2.VideoCapture(2)

while cap.isOpened():
    try:
        success, frame = cap.read()
        if not success:
            continue

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        relative_depth_map = estimate_relative_depth(rgb_frame)  # Get MiDaS depth
        cv2.imshow('relative_depth',relative_depth_map)
        # ===== AprilTag Detection =====
        tags = at_detector.detect(
            gray_frame, estimate_tag_pose=True,
            camera_params=(cam_mat[0, 0], cam_mat[1, 1], cam_mat[0, 2], cam_mat[1, 2]),
            tag_size=TAG_SIZE
        )

        apriltag_depth = None
        scaling_factor = None

        for tag in tags:
            # Compute depth using translation vector
            tVec = tag.pose_t.flatten()
            apriltag_depth = np.linalg.norm(tVec)

            # Draw tag bounding box
            for i in range(4):
                pt1 = tuple(tag.corners[i].astype(int))
                pt2 = tuple(tag.corners[(i + 1) % 4].astype(int))
                cv2.line(frame, pt1, pt2, (0, 255, 255), 2)

            # Display tag ID and depth
            top_left = tuple(tag.corners[0].astype(int))
            cv2.putText(frame, f"ID:{tag.tag_id} Dist:{round(apriltag_depth * 100, 2)} cm",
                       top_left, cv2.FONT_HERSHEY_PLAIN, 1.3, (0, 0, 255), 2)

            # Map tag corners to depth map size
            h, w = frame.shape[:2]
            depth_h, depth_w = relative_depth_map.shape[:2]
            corners = [(int(x * depth_w / w), int(y * depth_h / h)) for x, y in tag.corners]

            # Get median relative depth from the mapped corners
            marker_relative_depth = np.median([relative_depth_map[y, x] for x, y in corners])
            if marker_relative_depth > 0:
                scaling_factor = apriltag_depth / marker_relative_depth

        # ===== Hand Detection =====
        results = hands.process(rgb_frame)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                h, w, _ = frame.shape
                start = np.array([hand_landmarks.landmark[5].x, hand_landmarks.landmark[5].y])
                end = np.array([hand_landmarks.landmark[8].x, hand_landmarks.landmark[8].y])
                vector = end - start
                vector /= np.linalg.norm(vector)
                extended_tip = end + vector * 0.1

                # Scale fingertip coordinates to frame size
                extended_tip_pixel = (int(extended_tip[0] * w), int(extended_tip[1] * h))
                if 5 not in last_positions:
                    last_positions[5] = extended_tip_pixel

                smoothed_tip = (
                    int(last_positions[5][0] * (1 - SMOOTHING_FACTOR) + extended_tip_pixel[0] * SMOOTHING_FACTOR),
                    int(last_positions[5][1] * (1 - SMOOTHING_FACTOR) + extended_tip_pixel[1] * SMOOTHING_FACTOR)
                )
                last_positions[5] = smoothed_tip

                x, y = smoothed_tip
                roi_size = 50
                x1, y1 = max(0, x - roi_size), max(0, y - roi_size)
                x2, y2 = min(w, x + roi_size), max(0, min(h, y + roi_size))
                roi = frame[y1:y2, x1:x2]
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                edges = cv2.Canny(gray, 50, 150)
                contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                for cnt in contours:
                    cv2.drawContours(frame[y1:y2, x1:x2], [cnt], -1, (0, 255, 0), 2)
                # Map to depth map size
                
                x1_, y1_ = max(0, x - roi_size), max(0, y - roi_size)
                x2_, y2_ = min(w, x + roi_size), max(0, min(h, y + roi_size))

                # Compute absolute depth at fingertip
                rect_relative_depth = np.mean(relative_depth_map[y1_:y2_, x1_:x2_])
                rect_absolute_depth = rect_relative_depth * scaling_factor if scaling_factor else None

                # Display depth info
                if apriltag_depth:
                    cv2.putText(frame, f"Tag Depth: {round(apriltag_depth * 100, 2)} cm", (20, 30),
                               cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 2)
                    cv2.putText(frame, f"Rect Depth: {round(rect_absolute_depth * 100, 2) if rect_absolute_depth else 'N/A'} cm",
                               (20, 90), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 2)
        cv2.namedWindow("AprilTag & Hand Depth",cv2.WINDOW_NORMAL)
        cv2.imshow("AprilTag & Hand Depth", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except Exception as e:
        print(e)
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1743323705.932335   23560 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743323705.951054   28267 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650 Ti/PCIe/SSE2
W0000 00:00:1743323705.982507   28256 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743323706.004837   28260 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Using cache found in /home/abhinand/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /home/abhinand/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /home/abhinand/.cache/torch/hub/intel-isl_MiDaS_master
QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044fc0).
Cannot move to target thread (0x5ef399acfa50)

QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044fc0).
Cannot move to target thread (0x5ef399acfa50)

QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044fc0).
Cannot move to target thread (0x5ef399acfa50)

QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044fc0).
Cannot move to target thread (0x5ef399acfa50)

QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044fc0).
Cannot move to target thread (0x5ef399acfa50)

QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044

### Apriltag - midas - transformation

In [7]:

from Segment import Segmentation
from MiDaS_depth import MiDaS_depth
from ApriltagModule import ApriltagModule
import cv2
import numpy as np
from dt_apriltags import Detector
import time

root = os.getcwd()
calib_data_path = os.path.join(root, 'charuco_calib', 'calib_data', 'MultiMatrix.npz') 

segment = Segmentation()
depth = MiDaS_depth()
apriltag = ApriltagModule(calib_data_path=calib_data_path,family='tag36h11',tag_size=0.05)

cap = cv2.VideoCapture(2)
while cap.isOpened():
    try:
        success, frame = cap.read()
        if not success:
            continue
        set_time = time.time()
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_frame = cv2.resize(rgb_frame, (256, 256))
        relative_depth_map = depth.get_depthmap(rgb_frame)  # Get MiDaS depth
        relative_depth_map = cv2.resize(relative_depth_map, (frame.shape[1], frame.shape[0]))
        h, w, _ = frame.shape  # Get height and width before using them

        # ===== AprilTag Detection =====
        tags = apriltag.get_tags(frame)

        apriltag_depth = None
        scaling_factor = 1.0
        if len(tags)>0:
            scaling_factor = apriltag.get_scaling_factor(tags=tags,frame=frame,relative_depth_map=relative_depth_map)


            x,y = segment.get_smoothed_tip(frame)

            roi_size = 50
            x1, y1 = max(0, int(x - roi_size / 2)), max(0, int(y - roi_size / 2))
            x2, y2 = min(w, int(x + roi_size / 2)), min(h, int(y + roi_size / 2))
            roi = frame[y1:y2, x1:x2]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            #Canny edge detection
            gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 50, 150)
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for cnt in contours:
                cv2.drawContours(frame[y1:y2, x1:x2], [cnt], -1, (0, 255, 0), 2)

            # Compute absolute depth at fingertip
            rect_relative_depth = np.mean(relative_depth_map[y1:y2, x1:x2])
            rect_absolute_depth = rect_relative_depth * scaling_factor if scaling_factor else None
            
            # Display depth info
            if apriltag_depth:
                cv2.putText(frame, f"Tag Depth: {round(apriltag_depth * 100, 2)} cm", (20, 30),
                            cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 2)
                cv2.putText(frame, f"Rect Depth: {round(rect_absolute_depth * 100, 2) if rect_absolute_depth else 'N/A'} cm",
                            (20, 90), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 2)
                
            if apriltag_depth and rect_absolute_depth:

                # Compute 3D coordinates of the rectangle center (fingertip region center)
                rect_center_distorted = np.array([[x, y]], dtype=np.float32)
                rect_center = cv2.undistortPoints(rect_center_distorted, cam_mat, dist_coef)
                rect_center_homogeneous = np.array([rect_center[0][0][0], -rect_center[0][0][1], 1.0])

                # Scale with depth to get real-world 3D coordinates
                rect_center_3d = rect_center_homogeneous * rect_absolute_depth

                # Display coordinates relative to AprilTag
                cv2.putText(frame, f"X: {rect_center_3d[0]:.2f}, Y: {rect_center_3d[1]:.2f}, Z: {rect_center_3d[2]:.2f}",
                        (20, 120), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 255), 2)
                # cv2.putText(frame, f"X: {rect_center[0][0][0]:.2f}, Y: {-rect_center[0][0][1]:.2f}",
                #      (20, 120), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 255), 2)

                # === Plot optical center ===
                optical_center = (int(cam_mat[0, 2]), int(cam_mat[1, 2]))  # (cx, cy)
                cv2.circle(frame, optical_center, 5, (0, 0, 255), -1)  # Red dot for optical center
        # print("Time taken: ",time.time()-set_time)
        cv2.imshow("AprilTag & Hand Depth", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except Exception as e:
        print(e)
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1743325440.479251   23560 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743325440.496482   31373 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650 Ti/PCIe/SSE2
W0000 00:00:1743325440.527538   31361 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743325440.570838   31367 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Using cache found in /home/abhinand/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /home/abhinand/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /home/abhinand/.cache/torch/hub/intel-isl_MiDaS_master
QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044fc0).
Cannot move to target thread (0x5ef399acfa50)

QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044fc0).
Cannot move to target thread (0x5ef399acfa50)

QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044fc0).
Cannot move to target thread (0x5ef399acfa50)

QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044fc0).
Cannot move to target thread (0x5ef399acfa50)

QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044fc0).
Cannot move to target thread (0x5ef399acfa50)

QObject::moveToThread: Current thread (0x5ef399acfa50) is not the object's thread (0x5ef39a044

unsupported operand type(s) for -: 'NoneType' and 'float'


In [28]:
print(apriltag_depth)

None


#### Object selection

In [ ]:
def detect_apriltags(gray_frame):
    
    calib_data = np.load(calib_data_path)
    cam_mat = calib_data["camMatrix"]
    dist_coef = calib_data["distCoef"]
    tags = at_detector.detect(
            gray_frame, estimate_tag_pose=True,
            camera_params=(cam_mat[0, 0], cam_mat[1, 1], cam_mat[0, 2], cam_mat[1, 2]),
            tag_size=TAG_SIZE
        )

    apriltag_depth = None
    scaling_factor = None

    for tag in tags:
        # Compute depth using translation vector
        tVec = tag.pose_t.flatten()
        apriltag_depth = np.linalg.norm(tVec)
        rVec, _ = cv2.Rodrigues(tag.pose_R)

        # Define axis length (in meters)
        axis_length = 0.05

        # Define axis endpoints in 3D (relative to tag origin)
        axis_points = np.float32([
            [0, 0, 0],  # Origin
            [axis_length, 0, 0],  # X-axis
            [0, axis_length, 0],  # Y-axis
            [0, 0, -axis_length]  # Z-axis (negative since depth is into the screen)
        ]).reshape(-1, 3)

        # Project 3D points to 2D image plane
        img_pts, _ = cv2.projectPoints(axis_points, rVec, tVec, cam_mat, dist_coef)

        # Convert to integer for drawing
        img_pts = img_pts.astype(int)

        # Draw axes at the tag location
        origin = tuple(img_pts[0].ravel())
        cv2.line(frame, origin, tuple(img_pts[1].ravel()), (0, 0, 255), 2)  # X-axis (red)
        cv2.line(frame, origin, tuple(img_pts[2].ravel()), (0, 255, 0), 2)  # Y-axis (green)
        cv2.line(frame, origin, tuple(img_pts[3].ravel()), (255, 0, 0), 2)  # Z-axis (blue)

        # Label axes
        cv2.putText(frame, "X", tuple(img_pts[1].ravel()), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
        cv2.putText(frame, "Y", tuple(img_pts[2].ravel()), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
        cv2.putText(frame, "Z", tuple(img_pts[3].ravel()), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)

        # Draw tag bounding box
        for i in range(4):
            pt1 = tuple(tag.corners[i].astype(int))
            pt2 = tuple(tag.corners[(i + 1) % 4].astype(int))
            cv2.line(frame, pt1, pt2, (0, 255, 255), 2)

        # Display tag ID and depth
        top_left = tuple(tag.corners[0].astype(int))
        cv2.putText(frame, f"ID:{tag.tag_id} Dist:{round(apriltag_depth * 100, 2)} cm",
                    top_left, cv2.FONT_HERSHEY_PLAIN, 1.3, (0, 0, 255), 2)

        # Map tag corners to depth map size
        # Map tag corners to depth map size and clip to bounds
        h, w = frame.shape[:2]
        depth_h, depth_w = relative_depth_map.shape[:2]
        corners = [
            (
                int(np.clip(x * depth_w / w, 0, depth_w - 1)),
                int(np.clip(y * depth_h / h, 0, depth_h - 1))
            )
            for x, y in tag.corners
        ]

        # Get median relative depth from the mapped corners (only valid indices)
        marker_relative_depth = np.median([
            relative_depth_map[y, x]
            for x, y in corners
            if 0 <= x < depth_w and 0 <= y < depth_h
        ])

In [13]:

import cv2
import mediapipe as mp
import numpy as np
import os
import os
import torch
from ultralytics import FastSAM

from Segment import Segmentation
from MiDaS_depth import MiDaS_depth
from Apriltag import Apriltag

seg_model = FastSAM("FastSAM-s.pt")
root = os.getcwd()
calib_data_path = os.path.join(root, 'charuco_calib', 'calib_data', 'MultiMatrix.npz') 
segment = Segmentation()
depth = MiDaS_depth(calib_data_path)
apriltag = Apriltag(calib_data_path=calib_data_path,family='tag36h11',tag_size=0.05)

ROI_SIZE = 100
cap = cv2.VideoCapture(2)
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        continue
    set_time = time.time()
    x,y  = segment.get_smoothed_tip(frame)
    h, w, _ = frame.shape
    if  x is not None and y is not None:
        canny_frame = segment.draw_canny(center=(x,y),frame=frame,roi_size=ROI_SIZE)

        blurred_frame = cv2.GaussianBlur(canny_frame, (5, 5), 0)  # Apply mild blur
        seg_frame = seg_model.predict(blurred_frame,points=[x,y])[0]
        seg_frame_plot = cv2.resize(seg_frame.plot(conf=False,labels=False),(1920,1080))
        bboxes = seg_frame.boxes.xyxy.cpu().numpy()
        if len(bboxes) > 0:
            distances = []
            centers = []

            for box in bboxes:
                x1, y1, x2, y2 = box
                center_x, center_y = int((x1 + x2) / 2), int((y1 + y2) / 2)
                centers.append((center_x, center_y))

                # Ensure bounding box is inside the ROI
                if x1 < x < x2 and y1 < y< y2:
                    dist_sq = ((x - center_x) ** 2 + (y - center_y) ** 2)/(w*h)
                    distances.append(-dist_sq)  # Use negative squared distance for softmax
                else:
                    distances.append(-1e9)  # Very low score for out-of-ROI objects
            
            scores = np.exp(distances) / np.sum(np.exp(distances))
            for i, box in enumerate(bboxes):
                x1, y1, x2, y2 = map(int, box)
                score_text = f"{scores[i]:.3f}"

                # Draw bounding box
                cv2.rectangle(seg_frame_plot, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Draw score near the bounding box
                cv2.putText(seg_frame_plot, score_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        print("Time taken: ",time.time()-set_time)
        cv2.imshow("Segmented Object", seg_frame_plot)  
    else:            
        cv2.imshow("orig frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1742669417.964853    7862 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1742669417.981408   10593 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650 Ti/PCIe/SSE2
W0000 00:00:1742669418.005028   10579 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742669418.023714   10580 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Using cache found in /home/abhinand/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /home/abhinand/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /home/abhinand/.cache/torch/hub/intel-isl_MiDaS_master
QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to target thread (0x5ccd41e776a0)

QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to target thread (0x5ccd41e776a0)

QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to target thread (0x5ccd41e776a0)

QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to target thread (0x5ccd41e776a0)

QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to target thread (0x5ccd41e776a0)

QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb


0: 384x640 1 object, 17.4ms
Speed: 1.5ms preprocess, 17.4ms inference, 13.3ms postprocess per image at shape (1, 3, 384, 640)
Time taken:  0.27346158027648926

0: 384x640 1 object, 13.9ms
Speed: 3.0ms preprocess, 13.9ms inference, 13.2ms postprocess per image at shape (1, 3, 384, 640)
Time taken:  0.09233212471008301



QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to target thread (0x5ccd41e776a0)

QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to target thread (0x5ccd41e776a0)

QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to target thread (0x5ccd41e776a0)

QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to target thread (0x5ccd41e776a0)

QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to target thread (0x5ccd41e776a0)

QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to target thread (0x5ccd41e776a0)

QObject::moveToThread: Current thread (0x5ccd41e776a0) is not the object's thread (0x5ccd413fb380).
Cannot move to tar

0: 384x640 2 objects, 13.7ms
Speed: 2.0ms preprocess, 13.7ms inference, 15.0ms postprocess per image at shape (1, 3, 384, 640)
Time taken:  0.09610700607299805

0: 384x640 1 object, 13.0ms
Speed: 2.2ms preprocess, 13.0ms inference, 14.3ms postprocess per image at shape (1, 3, 384, 640)
Time taken:  0.09079718589782715

0: 384x640 2 objects, 13.7ms
Speed: 2.2ms preprocess, 13.7ms inference, 15.4ms postprocess per image at shape (1, 3, 384, 640)
Time taken:  0.08342337608337402

0: 384x640 2 objects, 12.6ms
Speed: 2.2ms preprocess, 12.6ms inference, 16.8ms postprocess per image at shape (1, 3, 384, 640)
Time taken:  0.08484673500061035

0: 384x640 2 objects, 12.3ms
Speed: 1.6ms preprocess, 12.3ms inference, 16.5ms postprocess per image at shape (1, 3, 384, 640)
Time taken:  0.08201813697814941

0: 384x640 3 objects, 13.4ms
Speed: 2.4ms preprocess, 13.4ms inference, 15.2ms postprocess per image at shape (1, 3, 384, 640)
Time taken:  0.08284783363342285

0: 384x640 1 object, 13.5ms
Speed: 

In [2]:
import os
print(os.listdir())


['.git', '.gitattributes', '.gitignore', '855564-hd_1920_1080_24fps.mp4', 'apriltag-imgs', 'apriltag.ipynb', 'april_tags', 'aruco_depth.ipynb', 'capture.jpg', 'charuco.pdf', 'charuco_calib', 'ChArUco_Marker.png', 'dpt_beit_large_512.pt', 'FastSAM-s.pt', 'first_frame.jpg', 'LICENSE', 'markers.pdf', 'midas_test.ipynb', 'mobile_sam.pt', 'pointing', 'rtdetr-l.pt', 'sam_vit_h_4b8939.pth', 'segmentation.ipynb', 'videos', 'yolo11n-pose.pt', 'yolo11n-seg.pt', 'yolo11s-pose.pt', 'yolov8s-seg.pt', 'ZoeDepth']
